<a id="header"></a>
<h1 style="background:Orange;color:white;padding-top:20px">
    <center>
        Sartorius - Cell Instance Segmentation
    </center>
</h1>
<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/30201/logos/header.png">

<a id="top"></a>
<div style="background: rgb(49,114,163);padding-bottom:10px">
<h1 style="text-align:center;color:white;">Table of Contents</h1>
</div>
<div style="padding:10px;text-align:center;background:rgba(0,0,0,0.08)">
<a href="#first" target="_self">Problem Statement<span style="color:white;float:right;background:rgba(0,0,0,0.5);padding:5px;border-radius:10px">1</span></a>
</div>

<div style="padding:10px;text-align:center;">
<a href="#second" target="_self">About Available Data<span style="color:white;float:right;background:rgba(0,0,0,0.5);padding:5px;border-radius:10px">2</span></a>
</div>

<div style="padding:10px;text-align:center;background:rgba(0,0,0,0.08)">
<a href="#third" target="_self" >[updated] Data Exploration<span style="color:white;float:right;background:rgba(0,0,0,0.5);padding:5px;border-radius:10px">3</span></a>
</div>

<div style="padding:10px;text-align:center">
<a href="#fourth" target="_self" >[updated] Vizualization of Images<span style="color:white;float:right;background:rgba(0,0,0,0.5);padding:5px;border-radius:10px">4</span></a>
</div>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from termcolor import colored

<a id="first"></a>
<h1 style="background:purple;color:white;padding-top:20px">
    <center>
        Problem Statement<span style="float:right"><a href="#top"><img src="https://www.clipartmax.com/png/middle/163-1630443_go-to-top-white-arrow-in-circle.png" height="40px" width="40px"></a></span>
    </center>
</h1>

<p style="color:red;font-weight:bold">This description has been copied from the competition page</p>
<p style="font-family:callibri;font-size:1.2em">Neurological disorders, including neurodegenerative diseases such as Alzheimer's and brain tumors, are a leading cause of death and disability across the globe. However, it is hard to quantify how well these deadly disorders respond to treatment. One accepted method is to review neuronal cells via light microscopy, which is both accessible and non-invasive. Unfortunately, segmenting individual neuronal cells in microscopic images can be challenging and time-intensive. Accurate instance segmentation of these cells, with the help of computer vision, could lead to new and effective drug discoveries to treat the millions of people with these disorders. Current solutions have limited accuracy for neuronal cells in particular. In internal studies to develop cell instance segmentation models, the neuroblastoma cell line SH-SY5Y consistently exhibits the lowest precision scores out of eight different cancer cell types tested. This could be because neuronal cells have a very unique, irregular and concave morphology associated with them, making them challenging to segment with commonly used mask heads.</p>
<p style="font-family:callibri;font-size:1.2em">In this competition, you’ll detect and delineate distinct objects of interest in biological images depicting neuronal cell types commonly used in the study of neurological disorders. More specifically, you'll use phase contrast microscopy images to train and test your model for instance segmentation of neuronal cells. Successful models will do this with a high level of accuracy.</p>
<h2 style="color:blue"> Evaluation Metric: </h2>
<p style="font-family:callibri;font-size:1.2em">
This competition is evaluated on the mean average precision at different intersection over union  thresholds. The IoU of a proposed set of object pixels and a set of true object pixels is calculated as:
<pre style="font-size:1.4em;text-align:center"> 𝐼𝑜𝑈(𝐴,𝐵) = 𝐴∩𝐵/𝐴∪𝐵</pre>

<p style="font-family:callibri;font-size:1.2em">The metric sweeps over a range of IoU thresholds, at each point calculating an average precision value. The threshold values range from 0.5 to 0.95 with a step size of 0.05: (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95). In other words, at a threshold of 0.5, a predicted object is considered a "hit" if its intersection over union with a ground truth object is greater than 0.5.

 <p style="font-family:callibri;font-size:1.2em">
At each threshold value , a precision value is calculated based on the number of true positives , false negatives , and false positives  resulting from comparing the predicted object to all ground truth objects:</p>
<pre style="font-size:1.4em;text-align:center">𝑇𝑃(𝑡) / (𝑇𝑃(𝑡)+𝐹𝑃(𝑡)+𝐹𝑁(𝑡))</pre>
<p style="font-family:callibri;font-size:1.2em">A true positive is counted when a single predicted object matches a ground truth object with an IoU above the threshold. A false positive indicates a predicted object had no associated ground truth object. A false negative indicates a ground truth object had no associated predicted object. The average precision of a single image is then calculated as the mean of the above precision values at each IoU threshold:

<p style="font-family:callibri;font-size:1.2em">
Lastly, the score returned by the competition metric is the mean taken over the individual average precisions of each image in the test dataset.
</p>

<a id="second"></a>
<h1 style="background:purple;color:white;padding-top:20px">
    <center>
        About Available Data<span style="float:right"><a href="#top"><img src="https://www.clipartmax.com/png/middle/163-1630443_go-to-top-white-arrow-in-circle.png" height="40px" width="40px"></a></span>
    </center>
</h1>

<p style="font-family:callibri;font-size:1.2em">This competition requires us to perform segmentation of neruonal cell in the images. The train and test images are provided in the respective folders. The annotations for each image is given in the train.csv file along with other info about the sample. The <b>train_semi_supervised</b> data has been provided incase the participants want to try a semi supervised approach to the problem. The <b>LIVECell_dataset_2021</b> data can be used as additional data for transfer learning.</p>
<h2>Files</h2>
<b>train.csv</b> - IDs and masks for all training objects. None of this metadata is provided for the test set.

<ul>
    <li>id - unique identifier for object</li>

<li>annotation - run length encoded pixels for the identified neuronal cell</li>

<li>width - source image width</li>

<li>height - source image height</li>

<li>cell_type - the cell line</li>

<li>plate_time - time plate was created</li>

<li>sample_date - date sample was created</li>

<li>sample_id - sample identifier</li>

<li>elapsed_timedelta - time since first image taken of sample</li>
</ul>

<b>sample_submission.csv</b> - a sample submission file in the correct format<br>

<b>train</b> - train images in PNG format<br>

<b>test</b> - test images in PNG format. Only a few test set images are available for download; the remainder can only be accessed by your notebooks when you submit.<br>

<b>train_semi_supervised</b> - unlabeled images offered in case you want to use additional data for a semi-supervised approach.<br>

<b>LIVECell_dataset_2021</b> - A mirror of the data from the LIVECell dataset. LIVECell is the predecessor dataset to this competition. You will find extra data for the SH-SHY5Y cell line, plus several other cell lines not covered in the competition dataset that may be of interest for transfer learning.<br>
    <p style="font-family:callibri;font-size:1.2em"> Now let's explore the given data and try to generate our own insights and some understanding.</p>

<a id="third"></a>
<h1 style="background:purple;color:white;padding-top:20px">
    <center>
       [updated] Data Exploration<span style="float:right"><a href="#top"><img src="https://www.clipartmax.com/png/middle/163-1630443_go-to-top-white-arrow-in-circle.png" height="40px" width="40px"></a></span>
    </center>
</h1>

In [ ]:
train = pd.read_csv("../input/sartorius-cell-instance-segmentation/train.csv")
train.head()

In [ ]:
tr_imgs = len(os.listdir("../input/sartorius-cell-instance-segmentation/train"))
te_imgs = len(os.listdir("../input/sartorius-cell-instance-segmentation/test"))
print(f"Size of the train data: {colored(train.shape[0], 'red')}")
print(f"Number of images in train set: {colored(tr_imgs, 'red')}")
print(f"Number of images in test set: {colored(te_imgs, 'red')}")

<p style="font-family:callibri;font-size:1.2em"> As you can see, although the number of train images is only 606, the size of the train set is sufficiently large. This is because a single image has multiple annotated objects. Since this is a code competition, the test set is hidden and one has to make notebook submissions. According to the host, the hidden test set has roughly 240 images. Let's see the number of annotations availble for each id.</p>

In [ ]:
plt.figure(figsize=(12,6));
vl = train["id"].value_counts().to_frame().reset_index().rename(columns={'index':'id',"id":"count"})
le = LabelEncoder()
vl["id"] = le.fit_transform(vl["id"])
sns.barplot(x="id",y = "count", data=vl);
plt.title("Number of samples for each Id");

<p style="font-family:callibri;font-size:1.2em"> While particular id may have upto roughly 800 annotations, majority of the ids have less than 100.</p>

<p style="font-family:callibri;font-size:1.2em"> Now let's have a look at different cell_types. As mentioned in the description, cell_types indicates the different types of cell line. In this data, there are three differnt cell types. Majority of the samples have cell type <b>shsy5y</b></p>

In [ ]:
plt.figure(figsize=(8,6));
sns.countplot(y="cell_type",data = train);
plt.title("Counts of Cell Types");

<a id="fourth"></a>
<h1 style="background:purple;color:white;padding-top:20px">
    <center>
        [updated] Vizualization of Images<span style="float:right"><a href="#top"><img src="https://www.clipartmax.com/png/middle/163-1630443_go-to-top-white-arrow-in-circle.png" height="40px" width="40px"></a></span>
    </center>
</h1>

<p style="color:red;font-weight:bold">This code has been copied from the <a href="https://www.kaggle.com/ihelon/cell-segmentation-run-length-decoding">notebook</a> by <a href="https://www.kaggle.com/ihelon">@ihelon</a></p>

In [ ]:
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()
    
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    
    img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
            
    for start, end in zip(starts, ends):
        img[start : end] = color
    
    return img.reshape(shape)

def plot_masks(image_id, colors=True):
    labels = train[train["id"] == image_id]["annotation"].tolist()
    cell_type = train[train["id"] == image_id]["cell_type"].tolist()
    cmap = {"shsy5y":(0,0,255),"astro":(0,255,0),"cort":(255,0,0)}

    if colors:
        mask = np.zeros((520, 704, 3))
        for label,cell_type in zip(labels,cell_type):
            c = cmap[cell_type]
            mask += rle_decode(label, shape=(520, 704, 3), color=c)
    else:
        mask = np.zeros((520, 704, 1))
        for label in labels:
            mask += rle_decode(label, shape=(520, 704, 1))
    mask = mask.clip(0, 1)

    image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(16, 32))
    plt.subplot(3, 1, 1)
    plt.imshow(image)
    plt.axis("off")
    plt.subplot(3, 1, 2)
    plt.imshow(image)
    plt.imshow(mask, alpha=0.5)
    plt.axis("off")
    plt.subplot(3, 1, 3)
    plt.imshow(mask)
    plt.axis("off")
    
    plt.show();

In [ ]:
plot_masks("ffdb3cc02eef", colors=False)

In [ ]:
plot_masks("0030fd0e6378", colors=False)

In [ ]:
def plot_masks_all(image_ids, colors=True):
    fig, ax = plt.subplots(len(image_ids),3,figsize=(16, 21))
    for idx,image_id in enumerate(image_ids):
        labels = train[train["id"] == image_id]["annotation"].tolist()
        cell_type = train[train["id"] == image_id]["cell_type"].tolist()
        cmap = {"shsy5y":(0,0,255),"astro":(0,255,0),"cort":(255,0,0)}

        if colors:
            mask = np.zeros((520, 704, 3))
            for label,cell_t in zip(labels,cell_type):
                c = cmap[cell_t]
                mask += rle_decode(label, shape=(520, 704, 3), color=c)
        else:
            mask = np.zeros((520, 704, 1))
            for label in labels:
                mask += rle_decode(label, shape=(520, 704, 1))
        mask = mask.clip(0, 1)

        image = cv2.imread(f"../input/sartorius-cell-instance-segmentation/train/{image_id}.png")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        ax[idx,0].imshow(image)
        ax[idx,0].set_title(f"Image {image_id}: cell_type {cell_t} Original")
        plt.axis("off")
        
        ax[idx,1].imshow(image)
        ax[idx,1].imshow(mask, alpha=0.5)
        ax[idx,1].set_title(f"Image with mask")
        plt.axis("off")
        
        ax[idx,2].imshow(mask)
        ax[idx,2].set_title(f"Only Mask")
        plt.axis("off")
    plt.tight_layout()
    fig.suptitle("Annotations Colored by Cell_Type: {blue: shsy5y, green: astro, red: cort}",fontsize=16)
    plt.show();

plot_masks_all(['042dc0e561a4', '04928f0866b0', '049f02e0f764', '085eb8fec206'])

# Work in Progress!!! 🚧🚧🚧